In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, evaluate, dump
from surprise import SVD, SVDpp, KNNBaseline, KNNWithMeans, CoClustering, GridSearch

In [2]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
sns.set_style("darkgrid")

In [4]:
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'])
df2 = pd.read_csv('combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'])
df3 = pd.read_csv('combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'])
df4 = pd.read_csv('combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'])

In [5]:
def add_movieID(df, movie_id):
    df_nan = pd.DataFrame(pd.isnull(df.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    df_nan = df_nan.reset_index()

    movie_np = []

    for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
        # numpy approach
        temp = np.full((1,i-j-1), movie_id)
        movie_np = np.append(movie_np, temp)
        movie_id += 1
        print(movie_id)

    # Account for last record and corresponding length
    # numpy approach
    last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
    movie_np = np.append(movie_np, last_record)

    print('Movie numpy: {}'.format(movie_np))
    print('Length: {}'.format(len(movie_np)))
    df = df[pd.notnull(df['Rating'])]

    df['Movie_Id'] = movie_np.astype(int)
    df['Cust_Id'] = df['Cust_Id'].astype(int)
    return df, movie_id

In [26]:
data = pd.DataFrame([])
tmp, movie_id = add_movieID(df1,1)
data = data.append(tmp)
tmp, movie_id = add_movieID(df2,movie_id)
data = data.append(tmp)
tmp, movie_id = add_movieID(df3,movie_id)
data = data.append(tmp)
tmp, movie_id = add_movieID(df4,movie_id)
data = data.append(tmp)

In [8]:
svdpp = SVDpp()

In [9]:
df = data

In [18]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']].sample(frac=0.02, replace=True), reader)
data.split(n_folds=3)

In [19]:
trainset = data.build_full_trainset()

In [25]:
probe = pd.read_csv("probe.txt",  names = ['Cust_Id'])
probe_movie = pd.DataFrame([])
Cust_Id = []
movie_id = []
currentMovie =0
index = 0

elem = 1
for i in probe.Cust_Id:
    if i[len(i) -1] == ":":
        elem = int(i[0:len(i)-1])
        print(elem)
        continue
    Cust_Id.append(int(i))
    movie_id.append(elem)
    #print(index,row)
probe_movie['Cust_Id'] = Cust_Id
probe_movie['Movie_Id'] = movie_id

In [13]:
merged_probe_set = pd.merge(probe_movie,df,on=['Cust_Id','Movie_Id'])

# #Parameter Tuning Code for epoch value = 5

In [14]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

In [20]:
grid_search = GridSearch(SVDpp, param_grid, measures=['RMSE', 'MAE'])

[{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}]


In [22]:
grid_search.evaluate(data)

------------
Parameters combination 1 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 1.0136
Mean MAE : 0.8251
------------
------------
Parameters combination 2 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 1.0171
Mean MAE : 0.8313
------------
------------
Parameters combination 3 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
------------
Mean RMSE: 0.9979
Mean MAE : 0.8079
------------
------------
Parameters combination 4 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
------------
Mean RMSE: 1.0023
Mean MAE : 0.8152
------------
------------
Parameters combination 5 of 8
params:  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 1.0016
Mean MAE : 0.8129
------------
------------
Parameters combination 6 of 8
params:  {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 1.0058
Mean MAE : 0.8199
------------
------------
Paramet

In [24]:
print(grid_search.evaluate)

defaultdict(<class 'list'>, {'rmse': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, 'mae': {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}})


In [23]:
svdpp.train(trainset)

In [24]:
cust_id = [];
index =1;
mae_sum = 0;
mse_sum = 0;
n = len(merged_probe_set)
for index,row in merged_probe_set.iterrows():
    user = row['Cust_Id']
    currentMovie = row['Movie_Id']
    pred = svdpp.predict(user, currentMovie, row['Rating'], verbose=False)
    diff = row['Rating']-pred.est
    mse_sum += diff*diff
    #print(diff)
mse = mse_sum/n
rmse = (mse_sum/n)**0.5
    
    # TODO: Try using model.test
    #pred = model.test(testData['Cust_Id'])
    #rmse(pred)
    #dump.dump('./dump_SVD', pred, model)
    
print(mse, rmse)

1.0972537568 1.04749880993
